In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/benjaminrike1/social_score_csr_reports.git

Cloning into 'social_score_csr_reports'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 218 (delta 0), reused 1 (delta 0), pack-reused 215
Receiving objects: 100% (218/218), 701.81 MiB | 42.86 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Checking out files: 100% (80/80), done.


In [ ]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 26.1 MB/s 


In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 17.0 MB/s 
     |████████████████████████████████| 462 kB 45.4 MB/s 


In [ ]:
pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 33.2 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
     |████████████████████████████████| 352 kB 73.0 MB/s 
     |████████████████████████████████| 90 kB 12.7 MB/s 
     |████████████████████████████████| 47.8 MB 81 kB/s 
     |████████████████████████████████| 1.2 MB 58.6 MB/s 
     |████████████████████████████████| 596 kB 55.6 MB/s 
     |████████████████████████████████| 99 kB 10.5 MB/s 
     |████████████████████████████████| 237 kB 12.2 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import pickle5 as pickle

In [ ]:
model_path = 'drive/MyDrive/model'
reloaded_model = tf.saved_model.load(model_path)


In [ ]:
def print_my_examples(inputs, results):
  print("Probabilities is on the format: neutral, positive, negative")
  result_for_printing = \
    [f'input: {inputs[i]:<30} : \n Probability: neutral: {results[i][0]:.2f}    positive: {results[i][1]:.2f}    negative: {results[i][2]:.2f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    #Facebook example
    'In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships',
    'In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers. ',
    #Cola examples
    'We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities.',
    'As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda',
    #Fake examples
    'Our diversity was reduced this year',
    'Football is a sport',
    #Cola examples
    'In 2020, 35.6% of leadership positions were held by women, up from 35.5% in 2019. ',
    'Zero fatalities occurred during the year',

    #SHV Energy
    'The launch of our cultural change CARE programme, which will embed ‘the seven health and safety habits’ within our organisation, was another success and', #positive
    '22% of our senior leadership team members are women', #negative?
    'Our Health and Safety Management System', #neutral
    'During the first quarter of 2021, we saw a spike in health and safety incidents', #negative
]
reloaded_results = tf.nn.softmax(reloaded_model(tf.constant(examples)))

print('Results from the model:')
print_my_examples(examples, reloaded_results)

Results from the model:
Probabilities is on the format: neutral, positive, negative
input: We lowered CO2 emissions this year : 
 Probability: neutral: 0.01    positive: 0.05    negative: 0.93
input: We are happy                   : 
 Probability: neutral: 0.80    positive: 0.07    negative: 0.13
input: We have seen an increase in the number of fatalities : 
 Probability: neutral: 0.00    positive: 0.02    negative: 0.98



In [ ]:
filename = 'social_score_csr_reports/reports_df.pickle'
with open(filename, 'rb') as fh:
  data = pickle.load(fh)

data.drop(labels='text', inplace=True)
data.head(5)

,Company,text,sentences
0,ABNAMRO,ABN AMRO Bank N.V.\n Integrated \nAnnual Repo...,[ABN AMRO Bank N.V. Integrated Annual Report ...
1,Agrifirm,2021 Annual Report\nBetter\nTogether\nA respon...,"[2021 , Annual Report, Better, Together, A res..."
2,AgroFair,SUSTAINABILITY \nREPORT 2020\n- 03 -\nCONTENT ...,"[SUSTAINABILITY REPORT 2020, - 03 -CONTENT IND..."
3,Amazon,All In: Staying the Course \non Our Commitmen...,[All In: Staying the Course on Our Commitment...
4,Armor,CORPORATE \nSOCIAL \nRESPONSIBILITY \nANNUAL ...,"[CORPORATE SOCIAL , RESPONSIBILITY , ANNUAL R..."


In [ ]:
import numpy as np
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

scores = [] #saving the individual company scores

# iterating over every company
for element in data['Company'].values:
  # extracting all sentences from the report
  features = data[data['Company']==element]['sentences'].values[0] 
  total_sentences = len(features)

  # variables for saving positive and negative count of sentences
  pos = 0
  neg = 0

  # hitting the model with sentences in chunks to avoid RAM overload
  for chunk in chunks(features, 128):
      # calculating softmax for every sentence
      results = tf.nn.softmax(reloaded_model(tf.constant(chunk)))
      # argmax to extract label
      argmax = tf.math.argmax(results, axis = 1).numpy()

      #counting pos sentences
      pos += np.count_nonzero(argmax==1)
  # assigning total scores realtive to number of sentences
  score = (pos)/total_sentences
  scores.append(score)

In [ ]:
data['scores'] = scores

In [ ]:
data[['Company', 'scores']].to_csv('drive/MyDrive/ml/social_scores.csv')

In [ ]:
for i,element in enumerate(scores):
  print("%s: %.3f"%(data['Company'].values[i],element))

ABNAMRO: 0.005
Agrifirm: 0.037
AgroFair: 0.056
Amazon: 0.310
Armor: 0.052
asr: 0.016
bam: 0.004
Bouwinvest: 0.083
Burggroup: 0.186
CocaCola: 0.089
Dataflex: 0.136
DS-Smith: 0.140
Essity: -0.011
Facebook: 0.203
Forbo: 0.154
FrieslandCampina: 0.028
Google: 0.146
GrandVision: 0.060
Grolsch: 0.140
heineken: 0.078
HEMA: 0.156
Henke: 0.174
HP: 0.127
IKEA: 0.178
IN: 0.047
Intel: 0.200
IOC: 0.279
KPN: 0.073
Microsoft: 0.129
NN-Group: 0.070
Optigroup: -0.002
Otrium: 0.200
PepsiCo: 0.204
Philips: 0.062
ProcterandGamble: 0.258
RAI: 0.001
Renewi: 0.115
Robeco: 0.213
Rockwool: 0.033
Royal-Avebe: -0.013
Shell: 0.099
SHVenergy: 0.161
TenneT: 0.088
Tesla: 0.091
Trivium: 0.172
Trust: 0.168
VanLanschot: -0.017
VPCAPITAL: 0.049
